# Heart Disease Exploration

Authors: Sophie Groenwold, Priyanka Banerjee

In [1]:
import numpy as np
import altair as alt
import pandas as pd

## Trying to use 1-4 target values

Scratch work.

In [2]:
kaggle_df = pd.read_csv('heart.csv')
# kaggle_df.head(10)
kaggle_df = kaggle_df.sort_values(by=['age'])
kaggle_df = kaggle_df.reset_index().drop('index', axis=1)
# print(kaggle_df['trestbps'].shape)
# print(kaggle_df.shape)

kaggle_df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
1,34,1,3,118,182,0,0,174,0,0.0,2,0,2,1
2,34,0,1,118,210,0,1,192,0,0.7,2,0,2,1
3,35,1,0,126,282,0,0,156,1,0.0,2,0,3,0
4,35,0,0,138,183,0,1,182,0,1.4,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,71,0,2,110,265,1,0,130,0,0.0,2,1,2,1
299,71,0,0,112,149,0,1,125,0,1.6,1,0,2,1
300,74,0,1,120,269,0,0,121,1,0.2,2,1,2,1
301,76,0,2,140,197,0,2,116,0,1.1,1,0,2,1


In [3]:
cleveland_df = pd.read_csv('processed_cleveland.csv')
# cleveland_df.head(10)
cleveland_df = cleveland_df.sort_values(by=['a'])
cleveland_df

,a,b,c,d,e,f,g,h,i,j,k,l,m,n
132,29.0,1.0,2.0,130.0,204.0,0.0,2.0,202.0,0.0,0.0,1.0,0.0,3.0,0
101,34.0,1.0,1.0,118.0,182.0,0.0,2.0,174.0,0.0,0.0,1.0,0.0,3.0,0
225,34.0,0.0,2.0,118.0,210.0,0.0,0.0,192.0,0.0,0.7,1.0,0.0,3.0,0
283,35.0,1.0,2.0,122.0,192.0,0.0,0.0,174.0,0.0,0.0,1.0,0.0,3.0,0
117,35.0,0.0,4.0,138.0,183.0,0.0,0.0,182.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,71.0,0.0,2.0,160.0,302.0,0.0,0.0,162.0,0.0,0.4,1.0,2.0,3.0,0
103,71.0,0.0,3.0,110.0,265.0,1.0,2.0,130.0,0.0,0.0,1.0,1.0,3.0,0
233,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0,0
257,76.0,0.0,3.0,140.0,197.0,0.0,1.0,116.0,0.0,1.1,2.0,0.0,3.0,0


In [4]:
# to view all the range of values for each feature

# for col in kaggle_df:
#     print(kaggle_df[col].unique())

# for col in cleveland_df:
#     print(cleveland_df[col].unique())

In [5]:
# age = a
# sex = b
# trestbps = d (but weird -- cleveland has an extra row)
# chol = e
# fbs = f
# thalach = h
# oldpeak = j

# kaggle values missing their counterpart:
# - cp
# - restecg
# - exang
# - slope
# - ca
# - thal
# - target (which is what we would've replaced)

# cleveland values missing their counterpart:
# - c
# - g
# - i
# - k
# - l
# - m
# - n (which is what we would've used in place of target)

In [6]:
# I used this to compare the different columns from each dataframe

# kaggle_df = kaggle_df.sort_values(by=['ca'])
# cleveland_df = cleveland_df.sort_values(by=['l'])
# print(kaggle_df['restecg'].unique().shape)
# print(cleveland_df['j'].unique().shape)
# print(kaggle_df['ca'].unique() == cleveland_df['l'].unique())

---

## PCA

Following similar procedures as from lab06.

In [7]:
# center and scale the data

kaggle_mean = np.mean(kaggle_df)
kaggle_centered = kaggle_df - kaggle_mean

In [8]:
u, s, vt = np.linalg.svd(kaggle_centered, full_matrices=False)
u.shape, s, vt.shape

((303, 14),
 array([902.30367687, 404.69484107, 303.53331729, 132.89232202,
         20.89507079,  17.58885715,  15.79391521,  10.58899348,
          9.24288227,   8.00324231,   7.56848802,   6.68148941,
          5.91763322,   5.53124326]),
 (14, 14))

In [9]:
# find and compare total variance found from:

# (1) data
total_variance_data = sum(np.var(kaggle_df, axis=0))

# (2) singular values
total_variance_singular_values = sum(np.square(s)) / len(kaggle_df)

print(total_variance_data)
print(total_variance_singular_values)

3594.5453800825626
3594.545380082563


In [10]:
# variance explained by first singular value
variance_explained_by_1st_pc = (s[0]**2 / len(kaggle_centered)) / (total_variance_singular_values)
variance_explained_by_1st_pc

0.7475131807660871

In [13]:
PC_num = []
i = 1
n = len(s)
while i <= n:
    PC_num.append(i)
    i += 1

explained_var = pd.DataFrame({
    'PC #': PC_num, 
    'Fraction of Variance Explained' : (s**2 / len(kaggle_centered)) / (total_variance_singular_values)
})

print(explained_varb)

# scree plot of variance explained by each PC
alt.Chart(explained_var, 
          title="Variance Explained by Principal Components"
).mark_bar(size=30).encode(
    alt.X('PC #:O'),
    alt.Y('Fraction of Variance Explained:Q')
).configure_axisX(labelAngle=0).properties(width=500)


    PC #  Fraction of Variance Explained
0      1                        0.747513
1      2                        0.150373
2      3                        0.084591
3      4                        0.016215
4      5                        0.000401
5      6                        0.000284
6      7                        0.000229
7      8                        0.000103
8      9                        0.000078
9     10                        0.000059
10    11                        0.000053
11    12                        0.000041
12    13                        0.000032
13    14                        0.000028


alt.Chart(...)

In [24]:
# relationships between PCs

P = pd.DataFrame(u @ np.diag(s))
P = P.rename(columns={0: "PC1", 1: "PC2", 2: "PC3", 3: "PC4", 4: "PC5", 5: "PC6", 6: "PC7",
                     7: "PC8", 8: "PC9", 9: "PC10", 10: "PC11", 11: "PC12", 12: "PC13", 13: "PC14"})
P['target'] = kaggle_df['target']

alt.Chart(P).mark_circle().encode(
    x = "PC1",
    y = "PC2",
    color = alt.Color('target:N', scale=alt.Scale(scheme='set1'))
)

alt.Chart(...)

In [28]:
# contributions of each feature kaggle_df on our PCs

def plot_pc(col_names, vt, k):
    """ 
    Plot how much each column of our data contributes 
    to each principal component and labels the rows of V^T.
    """
    df = pd.DataFrame({'v':vt[k, :], 'Column names':col_names})
    chart = alt.Chart(df).mark_bar().encode(
        x='Column names',
        y='v',
        opacity=alt.value(0.7)
    ).configure_axis(
        labelFontSize=12,
        titleFontSize=14
    ).configure_axisX(
        labelAngle = 0
    ).properties(width = 700)
    return chart

# contribution of each feature for PC1
plot_pc(list(kaggle_df.columns), vt, 0)

alt.Chart(...)

In [30]:
# contribution of each feature for PC2
plot_pc(list(kaggle_df.columns), vt, 1)

alt.Chart(...)

In [31]:
# contribution of each feature for PC3
plot_pc(list(kaggle_df.columns), vt, 2)

alt.Chart(...)

In [32]:
# contribution of each feature for PC4
plot_pc(list(kaggle_df.columns), vt, 3)

alt.Chart(...)